In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/input/siim-medical-images/overview.csv')
df.head(5)

In [ ]:
df['Age'].unique()

In [ ]:
import pydicom


file_path = '/kaggle/input/siim-medical-images/dicom_dir/ID_0000_AGE_0060_CONTRAST_1_CT.dcm'


ds = pydicom.dcmread(file_path)


patient_age = ds.PatientAge

print(f"Patient's Age: {patient_age}")


In [ ]:
import os
import pydicom
import numpy as np
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
import re

data_dir = '/kaggle/input/siim-medical-images/dicom_dir'


image_paths = os.listdir(data_dir)
lbp_features = []

age_ranges = []

radius = 1
n_points = 8 * radius
lbp = feature.local_binary_pattern
num_classes = 3  

def extract_sift_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)
    return keypoints, descriptors


for image_path in image_paths:
    dicom_file = pydicom.dcmread(os.path.join(data_dir, image_path))
    patient_age_str = dicom_file.PatientAge
    age_match = re.search(r'\d+', patient_age_str)
    if age_match:
        age = int(age_match.group())
        if 30 <= age < 50:
            age_ranges.append(0)
        elif 50 <= age < 70:
            age_ranges.append(1)
        elif 70 <= age <= 90:
            age_ranges.append(2)
        else:
            continue
    else:
        print(f"Cannot extract age from: {patient_age_str}")
    

    image_data = dicom_file.pixel_array  
    image_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data) * 255).astype(np.uint8)

    keypoints, descriptors = extract_sift_features(image_data)
    sift_features.append(descriptors)

    




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(lbp_features, age_ranges, test_size=0.2, random_state=42)


In [ ]:
y_test.count(0)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

svc = SVC(kernel='poly', C=1000)
svc.fit(X_train, y_train)


y_pred = svc.predict(X_test)


report = classification_report(y_test, y_pred, target_names=["30-50", "50-70", "70-90"])

print(report)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score 
svr = SVR(kernel='linear', C=10000.0)
svr.fit(X_train, y_train)


y_pred = svr.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')
r_squared = r2_score(y_test, y_pred)
print(f'R-squared (R^2): {r_squared}')

for i in range(len(y_test)):
    print(y_test[i],y_pred[i])


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=100)
rf_classifier.fit(X_train, y_train)


y_pred = rf_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt


data_dir = '/kaggle/input/siim-medical-images/dicom_dir'

dcm_files = [f for f in os.listdir(data_dir) if f.lower().endswith(".dcm")]


for dcm_file in dcm_files:
    dicom_file = pydicom.dcmread(os.path.join(data_dir, dcm_file))
    image_data = dicom_file.pixel_array  

    # Display the image
    plt.figure()
    plt.imshow(image_data, cmap='gray')
    plt.title(f'DICOM Image: {dcm_file}')
    plt.show()